In the Data folder file I showed that t>20 basically corresponds to noise. Here I want to check if the network performance improves if I throw out this noise.

In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]= '0' #, this way I would choose GPU 3 to do the work

import torch
import numpy as np
import time
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import zoom # for compressing images / only for testing purposes to speed up NN training
from scipy.fft import fft2, fftshift
from scipy.io import loadmat
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import torch.nn as nn

sys.path.append('../scripts')
sys.path.append('../models')

from sklearn.model_selection import train_test_split
import sys
from scipy.ndimage import rotate
import math
import random
from output_statistics import *

from tensorboard.backend.event_processing import event_accumulator

dimensions = 22 # for data reduction
trancuate_t = 96 # set this parameter to control at which time step you stop using the signal
import plotly.graph_objects as go
k = 1 # Set how many subsequent time steps you want to give to the network at once. Values allowed: 1, 2, 4, 8 (because it has to divide 8)

In [ ]:
test = True # display test error
train = True #display train error

# Naive_CNN_5Layer_possoin_r0_AF_3 = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low/5Layer/events.out.tfevents.1738583330.30adc46d0bc7.18369.0"
# Naive_CNN_5Layer_possoin_r0_AF_3_SC = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low_SC/5Layer/events.out.tfevents.1738583335.30adc46d0bc7.18411.0"

# Naive_CNN_10Layer_possoin_r0_AF_3 = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low/10Layer/events.out.tfevents.1738587247.f9c97f3588bc.883.0"
# Naive_CNN_10Layer_possoin_r0_AF_3_SC = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low_SC/10Layer/events.out.tfevents.1738587246.f9c97f3588bc.841.0"

# Interlacer_10Layer_possoin_r0_AF_3 = "../log_files/Interlacer_AF_3_possoin_r0_Full2Low/10Layer/events.out.tfevents.1738589774.f9c97f3588bc.1292.0"
# Interlacer_10Layer_possoin_r0_AF_3_SC = "../log_files/Interlacer_AF_3_possoin_r0_Full2Low_SC/10Layer/events.out.tfevents.1738590989.30adc46d0bc7.12850.0"

# Interlacer_20Layer_possoin_r0_AF_3 = "../log_files/Interlacer_AF_3_possoin_r0_Full2Low/20Layer/events.out.tfevents.1738683397.f9c97f3588bc.3432.0"
# Interlacer_20Layer_possoin_r0_AF_3_SC = "../log_files/Interlacer_AF_3_possoin_r0_Full2Low_SC/20Layer/events.out.tfevents.1738683439.f9c97f3588bc.3537.0"

# Naive_CNN_10Layer_possoin_r0_AF_3 = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low/10Layer/events.out.tfevents.1738683304.f9c97f3588bc.3319.0"
# Naive_CNN_10Layer_possoin_r0_AF_3_SC = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low_SC/10Layer/events.out.tfevents.1738683277.f9c97f3588bc.3215.0"

# Naive_CNN_10Layer_possoin_r0_AF_5 = "../log_files/Naive_CNN_3D_AF_5_possoin_r0_Full2Low/10Layer/events.out.tfevents.1738685403.30adc46d0bc7.14442.0"
# Naive_CNN_10Layer_possoin_r0_AF_5_SC = "../log_files/Naive_CNN_3D_AF_5_possoin_r0_Full2Low_SC/10Layer/events.out.tfevents.1738685435.30adc46d0bc7.14546.0"

# Naive_CNN_10Layer_possoin_r0_AF_2 = "../log_files/Naive_CNN_3D_AF_2_possoin_r0_Full2Low/10Layer/events.out.tfevents.1738739766.f9c97f3588bc.4464.0"
# Naive_CNN_10Layer_possoin_r0_AF_2_SC ="../log_files/Naive_CNN_3D_AF_2_possoin_r0_Full2Low_SC/10Layer/events.out.tfevents.1738739746.f9c97f3588bc.4360.0"

Naive_CNN_10Layer_Regular = "../log_files/Naive_CNN_3D_Regular_Full2Low/10Layer/events.out.tfevents.1738842127.f9c97f3588bc.7883.0"
Naive_CNN_10Layer_Regular_SC = "../log_files/Naive_CNN_3D_Regular_Full2Low_SC/10Layer/events.out.tfevents.1738841827.f9c97f3588bc.7777.0"

Naive_CNN_10Layer_Regular_96 = "../log_files/Naive_CNN_3D_AF_3_possoin_r0_Full2Low_96/10Layer/events.out.tfevents.1738849370.f9c97f3588bc.8622.0"

Naive_CNN_10Layer_Regular_96_NewMethod = "../log_files/Naive_CNN_3D_Regular_Full2Low_96/10Layer/events.out.tfevents.1739196933.f9c97f3588bc.226.0"
Naive_CNN_10Layer_Regular_96_NewMethod_SC = "../log_files/Naive_CNN_3D_Regular_Full2Low_SC_96/10Layer/events.out.tfevents.1739196996.f9c97f3588bc.333.0"
InterLacer_10Layer_Regular_96_NewMethod_SC = "../log_files/Interlacer_regular_Full2Low_SC/10Layer/events.out.tfevents.1739201439.f9c97f3588bc.1310.0"
InterLacer_10Layer_Regular_96_NewMethod = "../log_files/Interlacer_regular_Full2Low/10Layer/events.out.tfevents.1739205565.f9c97f3588bc.1441.0"

Naive_CNN_10Layer_Regular_20_NewMethod = "../log_files/Naive_CNN_3D_Regular_Full2Low_20/10Layer/events.out.tfevents.1739260921.f9c97f3588bc.1677.0"
Naive_CNN_10Layer_Regular_20_NewMethod_SC  = "../log_files/Naive_CNN_3D_Regular_Full2Low_SC_20/10Layer/events.out.tfevents.1739260905.f9c97f3588bc.1573.0"

Naive_CNN_10Layer_Regular_15_NewMethod = "../log_files/Naive_CNN_3D_Regular_Full2Low_15/10Layer/events.out.tfevents.1739271041.f9c97f3588bc.2004.0"
Naive_CNN_10Layer_Regular_15_NewMethod_SC  = "../log_files/Naive_CNN_3D_Regular_Full2Low_SC_15/10Layer/events.out.tfevents.1739270955.f9c97f3588bc.1897.0"

Naive_CNN_10Layer_Regular_96_NewMethod = "../log_files/Naive_CNN_3D_Regular_Full2Low_96/10Layer/events.out.tfevents.1739273666.f9c97f3588bc.2326.0"
Naive_CNN_10Layer_Regular_96_NewMethod_SC = "../log_files/Naive_CNN_3D_Regular_Full2Low_SC_96/10Layer/events.out.tfevents.1739273632.f9c97f3588bc.2222.0"

xzT_10_Layer = "../log_files/x-z-T-Naive_CNN_3D_Regular_Full2Low_96/10Layer/events.out.tfevents.1739347163.30adc46d0bc7.30002.0"
xzT_10_Layer_SC = "../log_files/x-z-T-Naive_CNN_3D_Regular_Full2Low_SC_96/10Layer/events.out.tfevents.1739347369.f9c97f3588bc.2632.0"

logdirs = [xzT_10_Layer, xzT_10_Layer_SC]

labels = ["xzT_10_Layer", "xzT_10_Layer_SC"]

# You can define the metrics you want to compare. For example:
my_metrics = [
    ("Loss Train", "Loss/Train"),
    ("Loss Test",  "Loss/Test"),
    ("PSNR Train", "Metric/PSNR /Train"),
    ("PSNR Test",  "Metric/PSNR /Test")
    #("SSIM Train", "Metric/SSIM /Train"),
    #("SSIM Test",  "Metric/SSIM /Test"),
]

compare_models(
    logdirs=logdirs,
    model_labels=labels,
    metrics=my_metrics,
    figsize=(12, 12),
    train = train,
    test = test
)

In [54]:
Inspect = "Loss/Test"   #SSIM Train     Loss/Test
logdir = Naive_CNN_10Layer_Regular_96_NewMethod

# Load the experiment data
ea = event_accumulator.EventAccumulator(logdir)
ea.Reload()  # Loads the log data

loss_data = {}

    # Extract Train Loss
if "Loss/Train" in ea.Tags()['scalars']:
    loss_train = ea.Scalars("Loss/Train")
    loss_data["Loss/Train"] = [(x.step, x.value) for x in loss_train]

    # Extract Test Loss
if "Loss/Test" in ea.Tags()['scalars']:
    loss_test = ea.Scalars("Loss/Test")
    loss_data["Loss/Test"] = [(x.step, x.value) for x in loss_test]
    
loss_test

[ScalarEvent(wall_time=1739273706.9442706, step=0, value=0.03710333630442619),
 ScalarEvent(wall_time=1739273746.6649919, step=1, value=0.03478090837597847),
 ScalarEvent(wall_time=1739273786.48767, step=2, value=0.03438471257686615),
 ScalarEvent(wall_time=1739273827.5177386, step=3, value=0.0341467522084713),
 ScalarEvent(wall_time=1739273867.360463, step=4, value=0.03404465317726135),
 ScalarEvent(wall_time=1739273907.6896002, step=5, value=0.03395453467965126),
 ScalarEvent(wall_time=1739273947.6363087, step=6, value=0.033880565315485),
 ScalarEvent(wall_time=1739273987.534284, step=7, value=0.03382444381713867),
 ScalarEvent(wall_time=1739274027.4028726, step=8, value=0.03375047445297241),
 ScalarEvent(wall_time=1739274067.3628976, step=9, value=0.033688317984342575),
 ScalarEvent(wall_time=1739274107.299365, step=10, value=0.033637866377830505),
 ScalarEvent(wall_time=1739274147.1568935, step=11, value=0.033621564507484436),
 ScalarEvent(wall_time=1739274187.4013364, step=12, val